In [1]:
import numpy as np
import os

# Define path to IS-RSA data and directory to save ISC vectors
is_rsa_path = '/Volumes/ARCHIVES/thesis_pipeline/data/IS_TR_RSA_ROIS/IS_TR_RSA_matrices_all_ROIs.npy'
isc_save_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors'
os.makedirs(isc_save_dir, exist_ok=True)

# Define ROI names (order should match the IS-RSA matrix structure)
roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']

# Load the IS-TR RSA matrix
is_rsa_matrix = np.load(is_rsa_path)
print(f"Loaded IS-TR RSA matrix with shape: {is_rsa_matrix.shape}")

# Loop through each region and compute the ISC vector
for i, roi in enumerate(roi_names):
    # Extract the IS-RSA data for the current ROI (shape: TRs x subjects x subjects)
    rsa_data = is_rsa_matrix[i]
    n_trs, n_subjects, _ = rsa_data.shape
    
    # Initialize an array to store the ISC values for each TR
    isc_vector = np.zeros(n_trs)
    
    # Compute ISC for each TR
    for tr in range(n_trs):
        # Get the subject-by-subject correlation matrix for the current TR
        subject_corr_matrix = rsa_data[tr]
        
        # Compute the mean of the upper triangle (excluding the diagonal) as ISC for the TR
        upper_triangle_indices = np.triu_indices(n_subjects, k=1)
        isc_value = np.mean(subject_corr_matrix[upper_triangle_indices])
        
        # Store the ISC value in the vector
        isc_vector[tr] = isc_value
    
    # Save the ISC vector for the current ROI
    isc_save_path = os.path.join(isc_save_dir, f'{roi}_ISC_vector.npy')
    np.save(isc_save_path, isc_vector)
    print(f"Saved ISC vector for {roi} with length {len(isc_vector)} at {isc_save_path}")


Loaded IS-TR RSA matrix with shape: (6, 1291, 8, 8)
Saved ISC vector for PTL with length 1291 at /Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors/PTL_ISC_vector.npy
Saved ISC vector for ATL with length 1291 at /Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors/ATL_ISC_vector.npy
Saved ISC vector for AG with length 1291 at /Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors/AG_ISC_vector.npy
Saved ISC vector for IFG with length 1291 at /Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors/IFG_ISC_vector.npy
Saved ISC vector for MFG with length 1291 at /Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors/MFG_ISC_vector.npy
Saved ISC vector for IFGorb with length 1291 at /Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors/IFGorb_ISC_vector.npy


In [7]:
import numpy as np
import os
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

# Paths to saved ISC vectors and HMM adjacency matrices
isc_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/ISC_vectors'
adjacency_dir = '/Volumes/ARCHIVES/thesis_pipeline/data/group_level_adjacency_matrices'
figures_dir = '/Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations'
os.makedirs(figures_dir, exist_ok=True)

roi_names = ['PTL', 'ATL', 'AG', 'IFG', 'MFG', 'IFGorb']
n_permutations = 5000
alpha = 0.05  # Significance level

# Function to compute mean adjacency vector from HMM adjacency matrix
def compute_hmm_adjacency_vector(adjacency_matrix):
    return np.mean(adjacency_matrix, axis=1)

for roi in roi_names:
    # Load the ISC vector for IS-RSA
    isc_path = os.path.join(isc_dir, f'{roi}_ISC_vector.npy')
    adjacency_path = os.path.join(adjacency_dir, f'{roi}_adjacency_matrix.npy')
    
    if os.path.exists(isc_path) and os.path.exists(adjacency_path):
        # Load ISC vector and HMM adjacency matrix
        isc_vector = np.load(isc_path)
        hmm_adjacency_matrix = np.load(adjacency_path)
        
        # Convert HMM adjacency matrix to vector
        hmm_adjacency_vector = compute_hmm_adjacency_vector(hmm_adjacency_matrix)
        
        # Ensure both vectors have the same length
        if len(isc_vector) != len(hmm_adjacency_vector):
            print(f"Length mismatch for {roi}: ISC vector length is {len(isc_vector)}, "
                  f"HMM adjacency vector length is {len(hmm_adjacency_vector)}. Skipping.")
            continue
        
        # Compute observed Spearman correlation
        observed_correlation, _ = spearmanr(isc_vector, hmm_adjacency_vector)
        print(f"Observed Spearman correlation for {roi}: {observed_correlation}")
        
        # Permutation testing
        permuted_correlations = []
        for _ in range(n_permutations):
            np.random.shuffle(isc_vector)  # Shuffle ISC vector
            permuted_correlation, _ = spearmanr(isc_vector, hmm_adjacency_vector)
            permuted_correlations.append(permuted_correlation)
        
        # Calculate p-value for two-tailed test
        permuted_correlations = np.array(permuted_correlations)
        p_value = (np.sum(np.abs(permuted_correlations) >= np.abs(observed_correlation)) + 1) / (n_permutations + 1)
        
        print(f"Permutation test p-value for {roi}: {p_value}")
        
        # Determine significance
        if p_value < alpha:
            print(f"The observed correlation for {roi} is statistically significant (p < {alpha}).")
        else:
            print(f"The observed correlation for {roi} is not statistically significant (p >= {alpha}).")
        
        # Plot histogram of permuted correlations
        plt.figure(figsize=(8, 6))
        plt.hist(permuted_correlations, bins=30, alpha=0.7, color='blue', label='Permuted Correlations')
        plt.axvline(observed_correlation, color='red', linestyle='dashed', linewidth=2, label='Observed Correlation')
        plt.axvline(np.percentile(permuted_correlations, 100 * alpha / 2), color='green', linestyle='dashed', linewidth=2, label=f'{alpha/2*100}% Threshold')
        plt.axvline(np.percentile(permuted_correlations, 100 * (1 - alpha / 2)), color='green', linestyle='dashed', linewidth=2)
        plt.xlabel('Spearman Correlation')
        plt.ylabel('Frequency')
        plt.title(f'Permutation Test for {roi}')
        plt.legend()
        plt.grid(True)
        
        # Save the plot
        plot_path = os.path.join(figures_dir, f'{roi}_permutation_test.png')
        plt.savefig(plot_path)
        plt.close()
        
        print(f"Saved permutation test plot for {roi} at {plot_path}")
    else:
        print(f"ISC vector or HMM adjacency matrix for {roi} not found.")


Observed Spearman correlation for PTL: -0.06978747303321318
Permutation test p-value for PTL: 0.014997000599880024
The observed correlation for PTL is statistically significant (p < 0.05).
Saved permutation test plot for PTL at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/PTL_permutation_test.png
Observed Spearman correlation for ATL: -0.04262954396332749
Permutation test p-value for ATL: 0.11737652469506099
The observed correlation for ATL is not statistically significant (p >= 0.05).
Saved permutation test plot for ATL at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/ATL_permutation_test.png
Observed Spearman correlation for AG: -0.10195307027494525
Permutation test p-value for AG: 0.0003999200159968006
The observed correlation for AG is statistically significant (p < 0.05).
Saved permutation test plot for AG at /Volumes/ARCHIVES/thesis_pipeline/figures/HMM_ISRSA_permutations/AG_permutation_test.png
Observed Spearman correlation for IFG: -0.0224